# Question: Does Oct4 extend the delay time i.e. cell plasticity?

### Statistical tests
**Notebook description/ method ** <br>

Previously, I have calculated the delay time differences between the GNE, GNEO1, and GNEO2 networks and the GN reference network, $\Delta\tau_{i -GN}$, where $i \in \{GNE, GNEO1, GNEO2\}$.

In this notebook, I perform statistical analysis on the $\Delta\tau_{i -GN}$ data.
For each condition (parameter, rel change), I compare the delay time difference.

First, I check if the distrition of delay time differences is a normal distribution.
Besides a plotting the distribution (graphical method), I perform a Shapiro-Wilk test, to acces whether the delay time differences are normally distributioned or not. The null hypothesis is: "The distribution of delay time diffrences follow a normal distribution." The choice if significance level $\alpha=0.05$ (common practice.)

If the normality assumption is valid, I perform a one-sided t-test with the null hypothesis: "The delay time differences between network i+1 is the same or smaller than the delay time differnces between network i." For example, if i = GNE, i+1 = GNEO1.

If the distributions of the delay time diffrences do not follow a normal distribution, I perform a one-sided Mann-Whitney U-test. The null hypothesis is the same: "The delay time differences between network i+1 is the same or smaller than the delay time differnces between network i."

In both cases, I use signicance level $\alpha=0.05$ (\*, reasonable confidence),$\alpha=0.01$,(\*\*)$\alpha=0.001$ (\*\*\*, high-confidence results) to test the strength of evidence. [note to self: i will only indicate the aesterics if the null hypothesis is rejected.]
**Result** <br>

Example of result (update when the analysis is done. this is just to rememeber what i am interested in): None of the distributions follow a normal distribution.


In [1]:
import scipy.stats as stats
from scipy.stats import shapiro
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## Do the distributions of the delay time differences follow a normal distribution?
#### perform a formal normality test (Shapiro-Wilk test)
H0: The sample is normally distributed

choice of significance level: $\sigma$=0.05 (widely accepted in practice)

Note on interpretation of the p-value:
If p-value > 0.05, the SW-test does not reject the null hypothesis, and the data is consistent with being normally distributed.

If p-value < 0.05, the SW-test does reject the null hypothesis, suggesting the data is not normally distributed.

In [2]:
sample_shapiro_results=False
if sample_shapiro_results:
    # 0. load data
    diff_tau_all = pd.read_csv("time_delay_diff_241010_v4.csv")
    
    # settings
    dataframe = diff_tau_all # renamed if I wanted to turn the following into a function. That was not needed.
    alpha_shapiro = 0.05
    date="241016"
    verbose=True # plot the distribution
    savefig=True # save the distribution

    # sample data
    shapiro_test_results = []

    for nets in dataframe["two_networks"].unique():
        for param in dataframe["parameter"].unique():
            for change in dataframe["rel change"].unique():
                
                # handle one condition at a time
                mask_networks = dataframe["two_networks"]==nets
                mask_param = dataframe["parameter"]==param
                mask_relchange = dataframe["rel change"]==change
                
                time_diff_data = dataframe[mask_networks & mask_param & mask_relchange]["time diff"]
                
                # perform shapiro wilk test
                stat, p_value = shapiro(time_diff_data)
                
                
                # check if we reject the null hypothesis (normality)
                if p_value > alpha_shapiro:
                    reject = False #Fail to reject H₀: Data looks normally distributed
                else:
                    reject = True  #Reject H₀: Data does not look normally distributed
                
                #print(nets, param, change, p_value, alpha_shapiro, reject)
                
                # plot the distribution
                if verbose:
                    Nbins = int(np.sqrt(len(time_diff_data)))
                    plt.figure()
                    freq, bins, patches = plt.hist(time_diff_data, bins=Nbins)
                    plt.xlabel(r"$\Delta\tau$")
                    plt.ylabel("Frequency")
                    plt.title(f"Nbins={Nbins}, p-value = {'{:.2e}'.format(p_value)}, reject = {reject} \n {param}, rel. change = {change}", fontsize=10)
                    plt.tight_layout()
                    if savefig:
                        plt.savefig(f"delay_time_diff_dist_{nets}_{param}_{change}_{date}.pdf", dpi=600)
                
                
                data = {"two_networks": nets,
                        "parameter":param,
                        "rel change":change,
                        "p_value": p_value,
                        "significance level":alpha_shapiro,
                        "reject": reject,
                       }
                
                shapiro_test_results.append(data)

    # Convert the list of dictionaries into a Pandas DataFrame
    df_shapiro_results = pd.DataFrame(shapiro_test_results)

    # Export the DataFrame to a CSV file
    df_shapiro_results.to_csv(f"shapiro_test_results_{date}.csv", index=False)

    print(f"Shapiro-Wilk test results saved to shapiro_test_results_{date}.csv")

# Only 2 out of 300 cases are normally distributed.

In [3]:
shapiro_results = pd.read_csv(f"shapiro_test_results_241016.csv")
shapiro_results[shapiro_results["reject"]==False]

,two_networks,parameter,rel change,p_value,significance level,reject
10,GNEvGN,alphaEO,0.2,0.064536,0.05,False
66,GNEvGN,KEG,2.0,0.200544,0.05,False


### Perform a one sided Mann-Whitney U test
I test the following
- Case A with $H_0$: The $\Delta\tau_{GNEO2-GN}$ are the same or smaller than the $\Delta\tau_{GNE-GN}$
- Case B with $H_0$: The $\Delta\tau_{GNEO1-GN}$ are the same or smaller than the $\Delta\tau_{GNE-GN}$
- Case C with $H_0$: The $\Delta\tau_{GNEO2-GN}$ are the same or smaller than the $\Delta\tau_{GNEO1-GN}$

In [7]:
from scipy.stats import mannwhitneyu

In [9]:
def MW_test(df_compare = GNEO1vGN,
            df_ref = GNEvGN,
            date="241016"
           ):
    # significance levels
    alpha_mannwhitney_005 = 0.05
    alpha_mannwhitney_001 = 0.01
    alpha_mannwhitney_0001=0.001


    MWresults = []
    # the function
    for param in df_compare["parameter"].unique():
        for change in df_compare["rel change"].unique():
            
            # prepare/ organise data
            mask_compare = (df_compare[df_compare["parameter"]==param]) & (df_compare[df_compare["rel change"]==change])
            mask_reference = (df_ref[df_ref["parameter"]==param]) & (df_ref[df_ref["rel change"]==change])
            
            df_compare_param_change = df_compare[mask_compare]["time diff"].reset_index(drop=True)
            df_ref_param_change = df_ref[mask_reference]["time diff"].reset_index(drop=True)
            
            # perfrom one sided mann whitney u test
            u_stat, p_value = mannwhitneyu(df_compare_param_change, df_ref_param_change, alternative='greater')
            
            # check if we can reject the null hypothesis
            
            if p_value > alpha_mannwhitney_005:
                reject_005 = False #Fail to reject H₀: no evidence that df_compare > df_ref
            elif p_value < alpha_mannwhitney_005:
                reject_005 = True
            
            if p_value > alpha_mannwhitney_001:
                reject_001 = False
            elif p_value < alpha_mannwhitney_001:
                reject_001 = True
            
            if p_value > alpha_mannwhitney_0001:
                reject_0001 = False
            elif p_value < alpha_mannwhitney_0001:
                reject_0001 = True
                
            data = {"df compare": df_compare["network_compare"].unique()[0],
                    "df ref": df_ref["network_compare"].unique()[0],
                    "parameter":param,
                    "rel change": change,
                    "p-value":p_value,
                    "reject 0.05":reject_005,
                    "reject 0.01":reject_001,
                    "reject 0.001": reject_0001
                   }
            MWresults.append(data)
            
    df_mannwhitney_results = pd.DataFrame(MWresults)

    # Export the DataFrame to a CSV file
    df_mannwhitney_results.to_csv(f"MW_results_{df_compare["network_compare"].unique()[0]}_{df_ref["network_compare"].unique()[0]}_{date}.csv", index=False)

            
    

SyntaxError: f-string: unmatched '[' (827577274.py, line 57)

In [8]:
# 0. load data
diff_tau_all = pd.read_csv("time_delay_diff_241010_v4.csv")

# group dataframe after two_networks
GNEvGN = diff_tau_all[diff_tau_all["two_networks"]=="GNEvGN"]
GNEO1vGN = diff_tau_all[diff_tau_all["two_networks"]=="GNEO1vGN"]
GNEO2vGN = diff_tau_all[diff_tau_all["two_networks"]=="GNEO2vGN"]

GNEvGN

,two_networks,network_ref,network_compare,parameter,rel change,condition 1,condition 2,time diff
0,GNEvGN,GN,GNE,alphaGN,0.2,False,False,1.026493
1,GNEvGN,GN,GNE,alphaGN,0.2,False,False,1.054603
2,GNEvGN,GN,GNE,alphaGN,0.2,False,False,1.078064
3,GNEvGN,GN,GNE,alphaGN,0.2,False,False,1.096036
4,GNEvGN,GN,GNE,alphaGN,0.2,False,False,0.808440
...,...,...,...,...,...,...,...,...
112695,GNEvGN,GN,GNE,KmaOE,5.0,True,True,1.673638
112696,GNEvGN,GN,GNE,KmaOE,5.0,True,True,1.734279
112697,GNEvGN,GN,GNE,KmaOE,5.0,True,True,1.812309
112698,GNEvGN,GN,GNE,KmaOE,5.0,True,True,1.948431


In [ ]:
u_stat, p_value = mannwhitneyu(delay_times_network, delay_times_reference, alternative='greater')
print(f"U-statistic: {u_stat}, One-sided p-value: {p_value}")

In [ ]:
# clean the code ---

In [ ]:
stats.probplot(data, dist="norm", plot=plt)

In [ ]:


# do not delete yet
# number of bins (Freedman-Diaconis Rule)

maxx=np.max(data)
minx=np.min(data)
Q3=data.quantile(0.75)
Q1=data.quantile(0.25)
IQR=Q3-Q1
N=len(data)
Nbins=(maxx-minx)/(2*IQR/N**(1/3))
#Nbins=int(np.sqrt(len(data)))
print(Nbins) # fewer bins than sqrt(len(data))

freq, bins, patches = plt.hist(data, bins=int(Nbins)) 

In [ ]:
data = [3.5, 7.2, 1.9, 5.4, 9.1]  # Unordered sample
stat, p_value = shapiro(data)

In [ ]:
p_value

### If normally distributed, perform a t-test one-sided

H0: The mean difference in delay time between (GNEO1 and GN) GNE and GN is (the same) zero or smaller (compared to the mean difference in delay time between GNE and GN.)

H1: The mean differnce in delay time between GNE and GN is larger than zero.


In [ ]:
## Assuming you have two arrays of delay times: delay_times_network and delay_times_reference
#t_stat, p_value = stats.ttest_ind(delay_times_network, delay_times_reference, alternative='greater')  # for one-sided

## If using a basic two-sided test:
## t_stat, p_value = stats.ttest_ind(delay_times_network, delay_times_reference)
## p_value /= 2  # adjust for one-sided

#print(f"T-statistic: {t_stat}, One-sided p-value: {p_value}")